# 영업팀 상품코드 생성 자동화 프로그램

## ERP에서 자료 가져오기

In [1]:
from sqlalchemy import create_engine
import cx_Oracle
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys

In [2]:
import pyautogui
import pyperclip # 한글 입력을 위한 클립보드 라이브러리
from timeit import default_timer as timer

In [3]:
import xlwings as xw

### DB 접속

In [4]:
sys.path.append('/settings')
import config

def connect_db(sid):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
        
    host = config.COMPANY_DB_CONFIG['host']
    port = config.COMPANY_DB_CONFIG['port']
    sid = config.COMPANY_DB_CONFIG['sid']

    serv = cx_Oracle.makedsn(host, port, sid=sid)

    cstr = 'oracle://{user}:{password}@{serv}'.format(
        user=config.COMPANY_DB_CONFIG['user'],
        password=config.COMPANY_DB_CONFIG['password'],
        serv=serv
    )

    conn = create_engine(cstr, convert_unicode=False, pool_recycle=-1, pool_size=5, echo=False)

    return conn

engine = connect_db('IVY')

### 조회조건 변수

In [5]:
# 실행파일로 만들 경우 이 변수들은 인풋으로 받아야 한다.
# N_season = '23N'
# F_season = '22F'
# S_season = '23S'
# agent = 'VA345'

#### N 시즌 입력

In [6]:
N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

In [7]:
while True:
    i = 0
    if len(N_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if N_season[-1] != 'N':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 N 입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if (N_season[0] >= '0' and N_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')
    
    if (N_season[1] >= '0' and N_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if i == 0:
        break

##### F 시즌 입력

In [8]:
F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

In [9]:
while True:
    i = 0
    if len(F_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if F_season[-1] != 'F':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 F 입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if (F_season[0] >= '0' and F_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')
    
    if (F_season[1] >= '0' and F_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if i == 0:
        break 

##### S 시즌 입력

In [10]:
S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

In [11]:
while True:
    i = 0
    if len(S_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if S_season[-1] != 'S':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 S 입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if (S_season[0] >= '0' and S_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')
    
    if (S_season[1] >= '0' and S_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 23S', default='23S')

    if i == 0:
        break 

##### 대리점 코드 입력

In [12]:
# VA34B  ,    VA3A9     ,    45347    , ASFDGA   
# VA345 ,    VA359     ,    GA347    ,   AS346  

while True:
    i = 0 # 탈출조건
    agents = pyautogui.prompt(title='대리점코드', text='대리점 코드를 입력하세요. 예 : VA345\
                                                       \n\n2개 이상의 대리점을 입력할 경우\
                                                       \n대리점코드 사이에 , 를 넣어주세요.\
                                                       \n공백은 넣지 않습니다.\
                                                       \n\n예 : VA345,VA346,VA347', default='')
    agents = agents.upper().split(',') # 공백제거

    for j, a in enumerate(agents):
        agents[j] = a.split()[0] # split 은 리턴값이 리스트이므로 0번째 값을 저장한다.


    for j, a in enumerate(agents):
        if len(a) != 5:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 대리점코드는 5자리로 입력하세요.\n\n입력한 값 : {agents[j]}')
            break
        if (a[0] >= '0' and a[0] <= '9') == True:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 1번째 글자는 문자입니다.\n\n입력한 값 : {agents[j]}')
            break
        if (a[1] >= '0' and a[1] <= '9') == True:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 2번째 글자는 문자입니다.\n\n입력한 값 : {agents[j]}')
            break
        if (a[2] >= '0' and a[2] <= '9') == False:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 3번째 글자는 숫자입니다.\n\n입력한 값 : {agents[j]}')
            break
        if (a[3] >= '0' and a[3] <= '9') == False:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 4번째 글자는 숫자입니다.\n\n입력한 값 : {agents[j]}')
            break
        if (a[4] >= '0' and a[4] <= '9') == False:
            i += 1
            pyautogui.alert(title='입력오류', text=f'{j+1}번째 대리점의 5번째 글자는 숫자입니다.\n\n입력한 값 : {agents[j]}')
            break

    if i == 0:
        break

agents

['VA345']

### 수행시간측정 시작

In [13]:
start_lab = timer()

### SQL문

- with open("파일이름.sql", "rt", encoding="utf8") as f:
- oracleSql = f.read()
- 
- 길이가 길 경우 파일로 넘겨준다.
- 
- 
- oracleSql = '''
- select agen_code,
-        rawtohex(utl_raw.cast_to_raw(agen_name)) agen_name
- from i_agen_t
- where 1=1

#### 한글변환 함수
- SQL문의 한글 컬럼에는 다음과 같이 함수로 처리
- rawtohex(utl_raw.cast_to_raw(변환할 컬럼))

In [14]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

#### SQL 본문
- 특약코드
    - ('C','D','H','I','L','R')
- 대리점코드
    - 강남점, 강동점, 분당점, 부천점, 만안점, 천안점  
    - ('VS455','VS022','VA156','VA338','VA146','VH017')
- 복종코드
    - 기타3 : cod_etc3 (동하복 복종)
        - F : 동복
        - S : 하복
        - T : 동하복공통
    - 기타4 : cod_etc4 (남녀 복종)
        - M : 남
        - F : 여
        - T : 남녀공통

##### 실제학교 함수 (시스템 조회 기준)

In [15]:
def product_code(N_season, F_season, S_season, agent):
    oracleSql = f'''
SELECT 'FM'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '0'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{N_season}' AS SEASON
        FROM   dual
        UNION ALL
        SELECT '{F_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT sch_code,
               sch_name,
               Substr(sch_code, 1, 1) SCH_AREA,
               Substr(sch_code, 2, 1) SCH_SEX
        FROM   i_sch_t,
               i_cod_t,
               i_tkyk_t,
               i_sch_com_t,
               i_sch_agen_t --대리점조회시 추가테이블 
        WHERE  sch_tkyk IN ( 'C', 'D', 'H', 'I',
                             'L', 'R' ) -- 특약 
               AND sca_code = sch_code
               AND sca_agen IN ( '{agent}' ) -- 대리점코드 
               AND sch_del_yn = 'Y' -- 사용여부 
               AND ( cod_gbn_code(+) = '02'
                     AND cod_code(+) = sch_cd_area_gb )
               AND tkyk_code(+) = sch_tkyk
               AND schc_code(+) = sch_com_cd
               AND Substr(sch_code, 2, 1) IN ( 'A', 'C', 'E', 'G' )) SCH,
       (SELECT cod_code,
               cod_name
        FROM   i_cod_t
        WHERE  cod_gbn_code = '01'
               AND del_yn = 'Y'
               AND cod_etc3 IN ( 'F', 'T' ) -- 시즌(동하복, 공통)
               AND cod_etc4 IN ( 'M', 'C' ) -- 남녀복종(남녀, 공통)
       ) BOK,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'FF'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '0'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{N_season}' AS SEASON
        FROM   dual
        UNION ALL
        SELECT '{F_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT sch_code,
               sch_name,
               Substr(sch_code, 1, 1) SCH_AREA,
               Substr(sch_code, 2, 1) SCH_SEX
        FROM   i_sch_t,
               i_cod_t,
               i_tkyk_t,
               i_sch_com_t,
               i_sch_agen_t --대리점조회시 추가테이블 
        WHERE  sch_tkyk IN ( 'C', 'D', 'H', 'I',
                             'L', 'R' ) -- 특약 
               AND sca_code = sch_code
               AND sca_agen IN ( '{agent}' ) -- 대리점코드 
               AND sch_del_yn = 'Y' -- 사용여부 
               AND ( cod_gbn_code(+) = '02'
                     AND cod_code(+) = sch_cd_area_gb )
               AND tkyk_code(+) = sch_tkyk
               AND schc_code(+) = sch_com_cd
               AND Substr(sch_code, 2, 1) IN ( 'B', 'D', 'F', 'H' ) -- 여자 
       ) SCH,
       (SELECT cod_code,
               cod_name
        FROM   i_cod_t
        WHERE  cod_gbn_code = '01'
               AND del_yn = 'Y'
               AND cod_etc3 IN ( 'F', 'T' ) -- 시즌(동하복, 공통)
               AND cod_etc4 IN ( 'F', 'C' ) -- 남녀복종(남녀, 공통)
       ) BOK,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'SM'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '0'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{S_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT sch_code,
               sch_name,
               Substr(sch_code, 1, 1) SCH_AREA,
               Substr(sch_code, 2, 1) SCH_SEX
        FROM   i_sch_t,
               i_cod_t,
               i_tkyk_t,
               i_sch_com_t,
               i_sch_agen_t --대리점조회시 추가테이블 
        WHERE  sch_tkyk IN ( 'C', 'D', 'H', 'I',
                             'L', 'R' ) -- 특약 
               AND sca_code = sch_code
               AND sca_agen IN ( '{agent}' ) -- 대리점코드 
               AND sch_del_yn = 'Y' -- 사용여부 
               AND ( cod_gbn_code(+) = '02'
                     AND cod_code(+) = sch_cd_area_gb )
               AND tkyk_code(+) = sch_tkyk
               AND schc_code(+) = sch_com_cd
               AND Substr(sch_code, 2, 1) IN ( 'A', 'C', 'E', 'G' )) SCH,
       (SELECT cod_code,
               cod_name
        FROM   i_cod_t
        WHERE  cod_gbn_code = '01'
               AND del_yn = 'Y'
               AND cod_etc3 IN ( 'S', 'T' ) -- 시즌(동하복, 공통)
               AND cod_etc4 IN ( 'M', 'C' ) -- 남녀복종(남녀, 공통)
       ) BOK,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'SF'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '0'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{S_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT sch_code,
               sch_name,
               Substr(sch_code, 1, 1) SCH_AREA,
               Substr(sch_code, 2, 1) SCH_SEX
        FROM   i_sch_t,
               i_cod_t,
               i_tkyk_t,
               i_sch_com_t,
               i_sch_agen_t --대리점조회시 추가테이블 
        WHERE  sch_tkyk IN ( 'C', 'D', 'H', 'I',
                             'L', 'R' ) -- 특약 
               AND sca_code = sch_code
               AND sca_agen IN ( '{agent}' ) -- 대리점코드 
               AND sch_del_yn = 'Y' -- 사용여부 
               AND ( cod_gbn_code(+) = '02'
                     AND cod_code(+) = sch_cd_area_gb )
               AND tkyk_code(+) = sch_tkyk
               AND schc_code(+) = sch_com_cd
               AND Substr(sch_code, 2, 1) IN ( 'B', 'D', 'F', 'H' ) -- 여자 
       ) SCH,
       (SELECT cod_code,
               cod_name
        FROM   i_cod_t
        WHERE  cod_gbn_code = '01'
               AND del_yn = 'Y'
               AND cod_etc3 IN ( 'S', 'T' ) -- 시즌(동하복, 공통)
               AND cod_etc4 IN ( 'F', 'C' ) -- 남녀복종(남녀, 공통)
       ) BOK,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB   
'''
    df = pd.read_sql_query(oracleSql, engine)
    df_temp = df['name'].copy()
    df['name'] = us7ascii_to_cp949(df_temp)
    
    return df

##### 공통학교 함수 (시스템 조회 기준)

In [16]:
def common_code(N_season, F_season, S_season, agent):
    oracleSql = f'''
SELECT 'FM'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '1'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{N_season}' AS SEASON
        FROM   dual
        UNION ALL
        SELECT '{F_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT A.comsch_season,
               A.comsch_school                        sch_code,
               B.sch_name,
               A.comsch_bokjong,
               Substr(A.comsch_school, 1, 1)          sch_area,
               Substr(A.comsch_school, 2, 1)          sch_sex,
               (SELECT Count(sca_code)
                FROM   i_sch_agen_t,
                       i_agen_t
                WHERE  sca_agen = agen_code
                       AND sca_code = A.comsch_school
                       AND sca_set_gb <> 'Y'
                       AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                          'L', 'R' )) agen_cnt,
               C.sca_code,
               C.sca_agen,
               D.agen_name,
               C.sca_set_gb,
               D.agen_tkyk,
               e.cod_code,
               e.cod_name
        FROM   i_stand_com_school A,
               i_sch_t B,
               i_sch_agen_t C,
               i_agen_t D,
               i_cod_t E
        WHERE  A.comsch_school = B.sch_code
               AND A.comsch_yn_gb = 'Y'
               AND sca_agen = agen_code
               AND sca_code = A.comsch_school
               AND comsch_bokjong = e.cod_code
               AND e.cod_gbn_code = '01'
               AND e.del_yn = 'Y'
               AND e.cod_etc3 IN ( 'F', 'T' ) -- 시즌(동하복, 공통)
               AND e.cod_etc4 IN ( 'M', 'C' ) -- 남녀복종(남녀, 공통)
               AND c.sca_set_gb <> 'Y'
               AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                  'L', 'R' )
               AND A.comsch_season = 'F'
               AND sca_agen IN ( '{agent}' ) -- 대리점코드
               AND Substr(comsch_school, 2, 1) IN ( 'A', 'C', 'E', 'G' )) SCH,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'FF'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '1'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{N_season}' AS SEASON
        FROM   dual
        UNION ALL
        SELECT '{F_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT A.comsch_season,
               A.comsch_school                        sch_code,
               B.sch_name,
               A.comsch_bokjong,
               Substr(A.comsch_school, 1, 1)          sch_area,
               Substr(A.comsch_school, 2, 1)          sch_sex,
               (SELECT Count(sca_code)
                FROM   i_sch_agen_t,
                       i_agen_t
                WHERE  sca_agen = agen_code
                       AND sca_code = A.comsch_school
                       AND sca_set_gb <> 'Y'
                       AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                          'L', 'R' )) agen_cnt,
               C.sca_code,
               C.sca_agen,
               D.agen_name,
               C.sca_set_gb,
               D.agen_tkyk,
               e.cod_code,
               e.cod_name
        FROM   i_stand_com_school A,
               i_sch_t B,
               i_sch_agen_t C,
               i_agen_t D,
               i_cod_t E
        WHERE  A.comsch_school = B.sch_code
               AND A.comsch_yn_gb = 'Y'
               AND sca_agen = agen_code
               AND sca_code = A.comsch_school
               AND comsch_bokjong = e.cod_code
               AND e.cod_gbn_code = '01'
               AND e.del_yn = 'Y'
               AND e.cod_etc3 IN ( 'F', 'T' ) -- 시즌(동하복, 공통)
               AND e.cod_etc4 IN ( 'F', 'C' ) -- 남녀복종(남녀, 공통)
               AND c.sca_set_gb <> 'Y'
               AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                  'L', 'R' )
               AND A.comsch_season = 'F'
               AND sca_agen IN ( '{agent}' ) -- 대리점코드
               AND Substr(comsch_school, 2, 1) IN ( 'B', 'D', 'F', 'H' )) SCH,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'SM'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '1'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{S_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT A.comsch_season,
               A.comsch_school                        sch_code,
               B.sch_name,
               A.comsch_bokjong,
               Substr(A.comsch_school, 1, 1)          sch_area,
               Substr(A.comsch_school, 2, 1)          sch_sex,
               (SELECT Count(sca_code)
                FROM   i_sch_agen_t,
                       i_agen_t
                WHERE  sca_agen = agen_code
                       AND sca_code = A.comsch_school
                       AND sca_set_gb <> 'Y'
                       AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                          'L', 'R' )) agen_cnt,
               C.sca_code,
               C.sca_agen,
               D.agen_name,
               C.sca_set_gb,
               D.agen_tkyk,
               e.cod_code,
               e.cod_name
        FROM   i_stand_com_school A,
               i_sch_t B,
               i_sch_agen_t C,
               i_agen_t D,
               i_cod_t E
        WHERE  A.comsch_school = B.sch_code
               AND A.comsch_yn_gb = 'Y'
               AND sca_agen = agen_code
               AND sca_code = A.comsch_school
               AND comsch_bokjong = e.cod_code
               AND e.cod_gbn_code = '01'
               AND e.del_yn = 'Y'
               AND e.cod_etc3 IN ( 'S', 'T' ) -- 시즌(동하복, 공통)
               AND e.cod_etc4 IN ( 'M', 'C' ) -- 남녀복종(남녀, 공통)
               AND c.sca_set_gb <> 'Y'
               AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                  'L', 'R' )
               AND A.comsch_season = 'S'
               AND sca_agen IN ( '{agent}' ) -- 대리점코드
               AND Substr(comsch_school, 2, 1) IN ( 'A', 'C', 'E', 'G' )) SCH,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB
UNION ALL
SELECT 'SF'                                      GRP,
       'IVYCLUB'                                 IVY_CD,
       Substr(season, 1, 2)                      YEAR,
       Substr(season, 3, 1)                      SS,
       sch_area                                  AREA,
       sch_sex                                   SEX,
       cod_code                                  BOKJONG,
       Decode(Substr(season, 3, 1), 'S', 'S',
                                    'F')         GUBUN,
       '1'                                       GUBUN2,
       sch_code                                  SCH_CD,
       Rawtohex(utl_raw.Cast_to_raw(sch_name
                                    ||' '
                                    ||cod_name)) NAME,
       size_code                                 SIZE_CD,
       size_code                                 PTN_CD,
       '100000'                                  TAG_PRICE,
       '100000'                                  SALE_PRICE,
       '100000'                                  ORIGIN_PRICE,
       '01'                                      DGN_CD,
       '0'                                       COLOR_CD,
       '01'                                      GH_CD,
       'JS'                                      HS_CD,
       '01'                                      GG_CD,
       ''                                        ITEM_CD,
       ''                                        BAR1,
       ''                                        BAR2,
       ''                                        HHY_CD,
       ''                                        STORY,
       ''                                        REMARK
FROM   (SELECT '{S_season}' AS SEASON
        FROM   dual) SEASON_TAB,
       (SELECT A.comsch_season,
               A.comsch_school                        sch_code,
               B.sch_name,
               A.comsch_bokjong,
               Substr(A.comsch_school, 1, 1)          sch_area,
               Substr(A.comsch_school, 2, 1)          sch_sex,
               (SELECT Count(sca_code)
                FROM   i_sch_agen_t,
                       i_agen_t
                WHERE  sca_agen = agen_code
                       AND sca_code = A.comsch_school
                       AND sca_set_gb <> 'Y'
                       AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                          'L', 'R' )) agen_cnt,
               C.sca_code,
               C.sca_agen,
               D.agen_name,
               C.sca_set_gb,
               D.agen_tkyk,
               e.cod_code,
               e.cod_name
        FROM   i_stand_com_school A,
               i_sch_t B,
               i_sch_agen_t C,
               i_agen_t D,
               i_cod_t E
        WHERE  A.comsch_school = B.sch_code
               AND A.comsch_yn_gb = 'Y'
               AND sca_agen = agen_code
               AND sca_code = A.comsch_school
               AND comsch_bokjong = e.cod_code
               AND e.cod_gbn_code = '01'
               AND e.del_yn = 'Y'
               AND e.cod_etc3 IN ( 'S', 'T' ) -- 시즌(동하복, 공통)
               AND e.cod_etc4 IN ( 'F', 'C' ) -- 남녀복종(남녀, 공통)
               AND c.sca_set_gb <> 'Y'
               AND agen_tkyk IN ( 'C', 'D', 'H', 'I',
                                  'L', 'R' )
               AND A.comsch_season = 'S'
               AND sca_agen IN ( '{agent}' ) -- 대리점코드
               AND Substr(comsch_school, 2, 1) IN ( 'B', 'D', 'F', 'H' )) SCH,
       (SELECT Trim(To_char(LEVEL, '09')) AS SIZE_CODE
        FROM   dual
        CONNECT BY LEVEL <= 15
        UNION ALL
        SELECT 'SS'
        FROM   dual
        UNION ALL
        SELECT 'BB'
        FROM   dual) SIZE_TAB 
'''
    df = pd.read_sql_query(oracleSql, engine)
    df_temp = df['name'].copy()
    df['name'] = us7ascii_to_cp949(df_temp)
    
    return df

### 데이터프레임 합치기 (원본데이터 작성)

In [17]:
df_code = pd.DataFrame([])

for agent in agents: # 입력받은 대리점들
    df_cd = product_code(N_season, F_season, S_season, agent)
    df_cd_common = common_code(N_season, F_season, S_season, agent) # 공통코드
    df_cd.insert(loc=0, column='agent', value=agent) # 대리점 코드를 맨 첫번째 열로 삽입함
    df_cd_common.insert(loc=0, column='agent', value=agent) # 공통코드
    df_code = pd.concat([df_code, df_cd, df_cd_common]) # 이전 데이터프레임에 합치기

 ### 데이터전처리 시작

#### 결측치 처리

In [18]:
df_code = df_code.fillna('')
df_code

,agent,grp,ivy_cd,year,ss,area,sex,bokjong,gubun,gubun2,...,color_cd,gh_cd,hs_cd,gg_cd,item_cd,bar1,bar2,hhy_cd,story,remark
0,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
1,VA345,FM,IVYCLUB,23,N,A,C,W,F,0,...,0,01,JS,01,,,,,,
2,VA345,FM,IVYCLUB,23,N,A,C,W,F,0,...,0,01,JS,01,,,,,,
3,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
4,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
217,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
218,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
219,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


#### 컬럼명 변경

In [19]:
df_code.columns = ['대리점',
                   '그룹',
                   '거래처코드',
                   '상품코드_년도코드',
                   '상품코드_시즌코드',
                   '상품코드_지역코드',
                   '상품코드_성별코드',
                   '상품코드_상품구분코드',
                   '상품코드_구분코드',
                   '상품코드_일반/공통복종구분',
                   '상품코드_학교코드',
                   '상품명',
                   '사이즈코드',
                   '사이즈패턴코드',
                   'TAG가',
                   '판매가',
                   '원가',
                   '디자이너코드',
                   '칼라코드',
                   '기획코드',
                   '행사코드',
                   '생산국가코드',
                   '아이템코드',
                   '바코드1',
                   '바코드2',
                   '혼용률',
                   'MD_STORY',
                   '봉제주의사항']
df_code

,대리점,그룹,거래처코드,상품코드_년도코드,상품코드_시즌코드,상품코드_지역코드,상품코드_성별코드,상품코드_상품구분코드,상품코드_구분코드,상품코드_일반/공통복종구분,...,칼라코드,기획코드,행사코드,생산국가코드,아이템코드,바코드1,바코드2,혼용률,MD_STORY,봉제주의사항
0,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
1,VA345,FM,IVYCLUB,23,N,A,C,W,F,0,...,0,01,JS,01,,,,,,
2,VA345,FM,IVYCLUB,23,N,A,C,W,F,0,...,0,01,JS,01,,,,,,
3,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
4,VA345,FM,IVYCLUB,23,N,A,A,W,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
217,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
218,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
219,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


In [20]:
df_code = df_code.sort_values(['대리점', '상품코드_년도코드', '상품코드_시즌코드', '상품코드_학교코드', '상품코드_상품구분코드', '사이즈코드'])

In [21]:
df_code

,대리점,그룹,거래처코드,상품코드_년도코드,상품코드_시즌코드,상품코드_지역코드,상품코드_성별코드,상품코드_상품구분코드,상품코드_구분코드,상품코드_일반/공통복종구분,...,칼라코드,기획코드,행사코드,생산국가코드,아이템코드,바코드1,바코드2,혼용률,MD_STORY,봉제주의사항
4050,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4084,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4118,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4152,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4186,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
217,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
218,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
220,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


In [22]:
def size_table():
    oracleSql = '''
    select size_bokjong, size_season, size_mfgbn,
        Rawtohex(utl_raw.Cast_to_raw(size_f1)) size_f1,
        Rawtohex(utl_raw.Cast_to_raw(size_f2)) size_f2,
        Rawtohex(utl_raw.Cast_to_raw(size_f3)) size_f3,
        Rawtohex(utl_raw.Cast_to_raw(size_f4)) size_f4,
        Rawtohex(utl_raw.Cast_to_raw(size_f5)) size_f5,
        Rawtohex(utl_raw.Cast_to_raw(size_f6)) size_f6,
        size_f7, size_f8, size_f9, size_f10, size_f11, size_f12,
        size_f13, size_t1, size_t2, size_t3, size_t4, size_t5, size_t6,
        size_t7, size_t8, size_t9, size_t10, size_t11, size_t12, size_t13,
        size_len, size_xjaepum_gb, size_f14, size_f15, size_t14, size_t15,
        size_pos_cloth_kind, size_unit_cd2, size_unit_cd1
    from i_size_t
    where 1=1
    '''
    df = pd.read_sql_query(oracleSql, engine)
    # print(df.columns)
    
    df_size = df[['size_bokjong', 'size_season', 'size_mfgbn', 'size_len', 'size_pos_cloth_kind',
                  'size_f1', 'size_f2','size_f3', 'size_f4', 'size_f5', 'size_f6', 'size_f7', 'size_f8','size_f9', 'size_f10',
                  'size_f11', 'size_f12', 'size_f13', 'size_f14', 'size_f15']].copy()
    df_size.columns = ['복종', '시즌', '성별', '사이즈길이', '포스코드',
                  '1호', '2호','3호', '4호', '5호', '6호', '7호', '8호','9호', '10호',
                  '11호', '12호', '13호', '14호', '15호']
    
    df_temp = df_size['1호'].copy()
    df_size['1호'] = us7ascii_to_cp949(df_temp)
    
    df_temp = df_size['2호'].copy()
    df_size['2호'] = us7ascii_to_cp949(df_temp)
    
    df_temp = df_size['3호'].copy()
    df_size['3호'] = us7ascii_to_cp949(df_temp)
    
    df_temp = df_size['4호'].copy()
    df_size['4호'] = us7ascii_to_cp949(df_temp)
    
    df_temp = df_size['5호'].copy()
    df_size['5호'] = us7ascii_to_cp949(df_temp)
    
    df_temp = df_size['6호'].copy()
    df_size['6호'] = us7ascii_to_cp949(df_temp)
    
    return df_size

In [23]:
df_size = size_table()
df_size

,복종,시즌,성별,사이즈길이,포스코드,1호,2호,3호,4호,5호,6호,7호,8호,9호,10호,11호,12호,13호,14호,15호
0,B,S,F,13,FB,80,85,90,95,100,105,110,115,120,125,130,135,140,None,None
1,J,F,F,13,FJ,80,85,90,95,100,105,110,115,120,125,130,135,140,None,None
2,J,F,M,13,MJ,80,85,90,95,100,105,110,115,120,125,130,135,140,None,None
3,P,F,F,15,FP,59,62,65,68,71,74,77,80,83,86,89,92,95,98,102
4,P,F,M,15,MP,62,65,68,71,74,77,80,83,86,89,92,95,98,102,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,G,F,2,13,FG,50,55,60,65,70,75,80,85,90,95,100,105,110,None,None
170,Q,S,F,15,FQ,59,62,65,68,71,74,77,80,83,86,89,92,95,98,102
171,J,F,A,0,AJ,36,37,38,39,40,41,42,43,44,45,None,None,None,None,None
172,G,F,1,13,MG,50,55,60,65,70,75,80,85,90,95,100,105,110,None,None


---

In [24]:
# df_code.to_clipboard()
# df_code.dtypes

In [25]:
# df_size.to_clipboard()

```python
df_code['상품코드_상품구분코드'] == 복종
df_code['상품코드_시즌코드'] == 시즌
df_code['상품코드_성별코드'] == 성별
```

In [26]:
# def mount_size() -> pd.DataFrame:
#     df1 = df_code.copy()
#     df1['오더'] = df1['상품코드_년도코드'] + df1['상품코드_시즌코드'] + df1['상품코드_학교코드'] + df1['상품코드_상품구분코드']
    
#     df_order_sum = pd.DataFrame()
#     for i, order in enumerate(df1['오더'].unique()):
        
#         df_order = df1[df1['오더'] == order].copy()
        
#         print(df_order[['사이즈코드', '사이즈패턴코드']])
        
        
#         # 복종
#         bok = df_order['상품코드_상품구분코드'].iloc[0]

#         # 시즌
#         if df_order['상품코드_시즌코드'].iloc[0] == 'S': # 시즌
#             season = 'S'
#         else:
#             season = 'F'

#         # 성별
#         if df_order['상품코드_성별코드'].iloc[0] in ('A', 'C'):
#             gbn = 'M'
#         elif df_order['상품코드_성별코드'].iloc[0] in ('B', 'D'):
#             gbn = 'F'
#         elif df_order['상품코드_성별코드'].iloc[0] == 'E':
#             gbn = '1'
#         elif df_order['상품코드_성별코드'].iloc[0] == 'F':
#             gbn = '2'
#         elif df_order['상품코드_성별코드'].iloc[0] == 'G':
#             gbn = '3'
#         elif df_order['상품코드_성별코드'].iloc[0] == 'H':
#             gbn = '4'
#         else:
#             print('성별에러발생')
#             sys.exit()
            
#         print('복종, 시즌, 성별 =', bok, season, gbn)
            
#         # 해당 사이즈 시리즈로 변경 (엑셀에 사이즈는 15 고정이라 일단 None 값까지 다 덮어버리고 나중에 중복제거 처리하는게 효율적)
#         sr_size = df_size[(df_size['복종'] == bok) & (df_size['시즌'] == season) & (df_size['성별'] == gbn) & (df_size['사이즈길이'] != '0')].iloc[0, 5:]
#         sr_size['SS'] = 'SS'
#         sr_size['BB'] = 'BB'
        
#         print(sr_size.iloc[3:-1])
        
#         df_order['사이즈코드'] = df_order['상품코드_상품구분코드'].copy()
#         df_order['사이즈패턴코드'] = sr_size
        
#         df_order_sum = pd.concat([df_order_sum, df_order])
        
#         if i == 2:
#             break
        
#     return df_order_sum

# df_code2 = mount_size()
# df_code2[['사이즈코드', '사이즈패턴코드']]

In [27]:
%%time

# 해당 사이즈 시리즈로 변경 (엑셀에 사이즈는 15 고정이라 일단 None 값까지 다 덮어버리고 나중에 중복제거 처리하는게 효율적)
# 사이즈코드 테이블 입히는 함수

def mount_size() -> pd.DataFrame:
    df1 = df_code.copy() # 본문
    df2 = df_size[(df_size['복종'].str.len()==1) & (df_size['성별'] != 'A') & (df_size['성별'] != 'B') & (df_size['사이즈길이'] != '0')].copy() # 사이즈 테이블 불러오기(허수제거)
    

    # df1['오더'] = df1['상품코드_년도코드'] + df1['상품코드_시즌코드'] + df1['상품코드_학교코드'] + df1['상품코드_상품구분코드']
    
    df1['시즌_2'] = df1['상품코드_시즌코드'].mask(~(df1['상품코드_시즌코드'] == 'S') , 'F').mask((df1['상품코드_시즌코드'] == 'S') , 'S')
    df1['성별_2'] = df1['상품코드_성별코드'].mask(df1['상품코드_성별코드'] == 'A', 'M').mask(df1['상품코드_성별코드'] == 'B', 'F').mask(df1['상품코드_성별코드'] == 'C', 'M').mask(df1['상품코드_성별코드'] == 'D', 'F').mask(df1['상품코드_성별코드'] == 'E', '1').mask(df1['상품코드_성별코드'] == 'F', '2').mask(df1['상품코드_성별코드'] == 'G', '3').mask(df1['상품코드_성별코드'] == 'H', '4')
    
    df1['사이즈코드'] = df1['상품코드_상품구분코드'].copy()
    
    for i in df2.index:
        bok = df2.at[i, '복종']
        ss = df2.at[i, '시즌']
        sex = df2.at[i, '성별']
        
        # print(i, bok, ss, sex)
        

        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '01')), '사이즈패턴코드'] = df2.at[i, '1호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '02')), '사이즈패턴코드'] = df2.at[i, '2호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '03')), '사이즈패턴코드'] = df2.at[i, '3호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '04')), '사이즈패턴코드'] = df2.at[i, '4호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '05')), '사이즈패턴코드'] = df2.at[i, '5호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '06')), '사이즈패턴코드'] = df2.at[i, '6호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '07')), '사이즈패턴코드'] = df2.at[i, '7호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '08')), '사이즈패턴코드'] = df2.at[i, '8호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '09')), '사이즈패턴코드'] = df2.at[i, '9호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '10')), '사이즈패턴코드'] = df2.at[i, '10호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '11')), '사이즈패턴코드'] = df2.at[i, '11호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '12')), '사이즈패턴코드'] = df2.at[i, '12호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '13')), '사이즈패턴코드'] = df2.at[i, '13호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '14')), '사이즈패턴코드'] = df2.at[i, '14호']
        df1.loc[((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '15')), '사이즈패턴코드'] = df2.at[i, '15호']

        
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '01')), df2.iloc[i, 5])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '02')), df2.iloc[i, 6])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '03')), df2.iloc[i, 7])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '04')), df2.iloc[i, 8])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '05')), df2.iloc[i, 9])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '06')), df2.iloc[i, 10])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '07')), df2.iloc[i, 11])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '08')), df2.iloc[i, 12])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '09')), df2.iloc[i, 13])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '10')), df2.iloc[i, 14])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '11')), df2.iloc[i, 15])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '12')), df2.iloc[i, 16])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '13')), df2.iloc[i, 17])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '14')), df2.iloc[i, 18])
        # df1['사이즈패턴코드'] = df1['사이즈패턴코드'].mask(((df1['사이즈코드'] == bok) & (df1['시즌_2'] == ss) & (df1['성별_2'] == sex) & (df1['사이즈패턴코드'] == '15')), df2.iloc[i, 19])
        
        
    df1.drop(['시즌_2', '성별_2'] ,axis=1, inplace=True)
    df1['사이즈패턴코드'] = df1['사이즈패턴코드'].str.strip()
    df1 = df1.reset_index(drop=True)
        
    return df1

df_code2 = mount_size()
# len(df_code2)
# df_code2.to_clipboard()
df_code2

# 7160
# 1680
# 5480

CPU times: total: 25.5 s
Wall time: 25.5 s


,대리점,그룹,거래처코드,상품코드_년도코드,상품코드_시즌코드,상품코드_지역코드,상품코드_성별코드,상품코드_상품구분코드,상품코드_구분코드,상품코드_일반/공통복종구분,...,칼라코드,기획코드,행사코드,생산국가코드,아이템코드,바코드1,바코드2,혼용률,MD_STORY,봉제주의사항
0,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
1,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
2,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
3,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46898,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46899,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46900,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46901,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


In [28]:
# sys.exit()

In [29]:
df_code2 = df_code2.drop(df_code2[df_code2['사이즈패턴코드'].isnull()].index).copy()
df_code2

,대리점,그룹,거래처코드,상품코드_년도코드,상품코드_시즌코드,상품코드_지역코드,상품코드_성별코드,상품코드_상품구분코드,상품코드_구분코드,상품코드_일반/공통복종구분,...,칼라코드,기획코드,행사코드,생산국가코드,아이템코드,바코드1,바코드2,혼용률,MD_STORY,봉제주의사항
0,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
1,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
2,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
3,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46898,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46899,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46900,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46901,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


In [30]:
df_code2 = df_code2.drop(df_code2[df_code2['사이즈패턴코드'].isin(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15'])].index).copy()
df_code2

,대리점,그룹,거래처코드,상품코드_년도코드,상품코드_시즌코드,상품코드_지역코드,상품코드_성별코드,상품코드_상품구분코드,상품코드_구분코드,상품코드_일반/공통복종구분,...,칼라코드,기획코드,행사코드,생산국가코드,아이템코드,바코드1,바코드2,혼용률,MD_STORY,봉제주의사항
0,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
1,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
2,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
3,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
4,VA345,FM,IVYCLUB,22,F,A,A,C,F,0,...,0,01,JS,01,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46898,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46899,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46900,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,
46901,VA345,SM,IVYCLUB,23,S,D,C,Q,S,1,...,0,01,JS,01,,,,,,


---

#### ** 공통코드 기입 관련 코드

---

- 공통코드 엑셀 시트 기입양식 예시

|대리점코드|실학교코드|공통학교코드|복종|
|:---:|:---:|:---:|:---:|
|VA146|AD754|DC297|P|
|VA146|AD757|DC297|P|
|VA345|AD662|DC297|P|

In [31]:
# # 아마도 학교와 공통학교는 dict 형태로 받아야 할 것 같다.

# agen_lst = ['VA146', 'VA345']
# sch_lst = ['AD754', 'AD757', 'AD662']
# bok_lst = ['P', 'J']
# common_l = 'DC297'

# def common_update(agen, sch, bok, common):
#     cond1 = df_code['대리점'] == agen
#     cond2 = df_code['상품코드_학교코드'] == sch
#     cond3 = df_code['상품코드_상품구분코드'] == bok
    
#     comm_name = df_code[df_code['상품코드_학교코드'] == 'DC297'].iat[0, 11] # 11번째 열이 '상품명'이다. at로 하면 에러남. 원인불명.
#     # comm_name = df_code[df_code['상품코드_학교코드'] == 'DC297']['상품명'].values[0] # 위와 동일한 코드
    
#     df_code['상품코드_일반/공통복종구분'] = df_code['상품코드_일반/공통복종구분'].mask(cond1 & cond2 & cond3, '1') # 공통 1찍기
#     df_code['상품명'] = df_code['상품명'].mask(cond1 & cond2 & cond3, comm_name) # 같은 조건으로 상품명 바꾸기
    
# for agen in agen_lst:
#     for sch in sch_lst:
#         for bok in bok_lst:
#             common_update(agen, sch, bok, common_l)
#             print(agen, sch, bok, common_l)
            
# df_code[(df_code['대리점'] == 'VA146') & (df_code['상품코드_학교코드'] == 'AD754') & (df_code['상품코드_상품구분코드'] == 'J')]['상품명']

#### 원본 데이터 CSV로 저장

In [32]:
df_code2.to_csv(f'{agents}.csv', index=False, encoding='CP949')

In [33]:
# sys.exit()

## 엑셀파일 작업 시작

### 엑셀 인스턴스 생성

In [34]:
app = xw.App(visible=True) # 실행과정 보이게
# app = xw.App(visible=False) # 실행과정 안보이게

### 반복문 작업 (1단 : 대리점 -> 2단 : 학교코드)

In [35]:
log_text = ''

for agent in agents:
    
    log_text += f'{agent} 통합문서 생성중입니다.\n'
    print(f'{agent} 통합문서 생성중입니다.')

    wb = xw.books.add() # 워크북 객체 생성
    
    df_agent = df_code2[df_code2['대리점'] == agent].copy() # 대리점 별 데이터프레임 분할
    df_agent = df_agent.drop(['대리점', '그룹'], axis=1) # 필요없는 열 삭제
    df_agent = df_agent.sort_values(['상품코드_년도코드', '상품코드_시즌코드', '상품코드_학교코드', '상품코드_상품구분코드', '사이즈코드'])
    
    sch_count = len(df_agent['상품코드_학교코드'].unique())
    log_text += f'총 {sch_count} 개의 학교가 있습니다.\n'
    print('총' , sch_count, '개의 학교가 있습니다.')
          
    for sch_code in (df_agent['상품코드_학교코드'].unique()): # 잘라낸 데이터프레임에서 학교코드 반복문 실행
        
        log_text += f'{sch_code} '
        print(sch_code, end=' ')
        
        sh1 = wb.sheets.add() # 새 시트 생성
        sh1.name = sch_code # 학교코드로 시트이름 지정
        # 형식 고정 (0 -> 00)
        sh1.range('K:L').number_format = '00'
        sh1.range('P:P').number_format = '00'
        sh1.range('R:R').number_format = '00'
        sh1.range('T:T').number_format = '00'
        sh1['A1'].options(index=False, dtype=str).value = df_agent[df_agent['상품코드_학교코드'] == sch_code] # 해당학교의 시트내용 복사

    log_text += '\n\n'
    print()
    print()

    sh1 = wb.sheets('Sheet1') # 첫 시트 지정
    sh1.delete() # 첫 시트 삭제
    wb.save(f'{agent}.xlsx') # 대리점 코드로 저장
    wb.close()

VA345 통합문서 생성중입니다.
총 67 개의 학교가 있습니다.
AA045 AA073 AA074 AA075 AA076 AA080 AA081 AA086 AA265 AA309 AA455 AA551 AA556 AA710 AA768 AA965 AA973 AA974 AB079 AB080 AB081 AB083 AB084 AB088 AB090 AB094 AB202 AB235 AB380 AB473 AB474 AB481 AB595 AB610 AC041 AC059 AC062 AC063 AC069 AC121 AC123 AC255 AC439 AC440 AC636 AC637 AC719 AC868 AC869 AC908 AD054 AD055 AD058 AD061 AD112 AD130 AD422 AD423 AD613 AD657 DC035 DC106 DC112 DC115 DD047 DD048 DC297 



### 액셀 인스턴스 종료

In [36]:
app.kill()

### 수행시간측정 종료

In [37]:
end_lab = timer()

In [38]:
pyautogui.alert(title='수행시간 측정', text=f'시작시각 : {start_lab}\n종료시각 : {end_lab}\n\n총 수행시간 : {end_lab-start_lab} 초')

'OK'

In [39]:
pyautogui.alert(title='작업로그', text=log_text)

'OK'

In [40]:
pyautogui.alert(title='프로그램 완료', text='지루한 작업이 모두 끝났습니다. \n수고하셨습니다.')

'OK'